Feature Engineering

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:

print("Loading energy data...")
df = pd.read_csv('cleaned_energy_data.csv')
df['INTERVAL_DATE'] = pd.to_datetime(df['INTERVAL_DATE'])

#time columns
time_cols = [col for col in df.columns if '.' in col and '-' in col]
all_icps = df['ICP_IDENTIFIER'].unique()

print(f"Loaded {len(df)} records for {len(all_icps)} households")
print(f"Date range: {df['INTERVAL_DATE'].min()} to {df['INTERVAL_DATE'].max()}")

# Convert time intervals from kWh to kW 
print("Converting time intervals from kWh to kW...")
df.loc[:, time_cols] = df[time_cols] * 2 

Loading energy data...
Loaded 109189 records for 118 households
Date range: 2022-01-01 00:00:00 to 2024-09-02 00:00:00
Converting time intervals from kWh to kW...


In [3]:
# Load room data from CSV
room_data = pd.read_csv('uniflats.csv')
room_data.columns = ['Number', 'Street', 'ICP', 'Rooms', 'Corresponding', 'List']

# Clean and prepare room data
room_data = room_data[['ICP', 'Rooms']].dropna()
room_data['ICP'] = room_data['ICP'].astype(str)

# Merge with energy data
df = df.merge(room_data[['ICP', 'Rooms']], 
              left_on='ICP_IDENTIFIER', 
              right_on='ICP', 
              how='left')

# Clean up extra ICP column
df = df.drop(columns=['ICP'])
print(f"Room data merged successfully.")
print(df[['ICP_IDENTIFIER', 'Rooms']].head())

Room data merged successfully.
    ICP_IDENTIFIER  Rooms
0  0000002795DE203     11
1  0000002795DE203     11
2  0000002795DE203     11
3  0000002795DE203     11
4  0000002795DE203     11


In [4]:
print("Loading weather data...")
# Load the daily weather data
daily_weather = pd.read_csv('weather_daily_aggregated.csv')
daily_weather['DATETIME'] = pd.to_datetime(daily_weather['DATETIME'])
daily_weather['DATE'] = daily_weather['DATETIME'].dt.date

# Get energy date range 
energy_start = df['INTERVAL_DATE'].min().date()
energy_end = df['INTERVAL_DATE'].max().date()
print(f"Energy data date range: {energy_start} to {energy_end}")

# Filter weather data to match energy date range
daily_weather = daily_weather[(daily_weather['DATE'] >= energy_start) & 
                             (daily_weather['DATE'] <= energy_end)]

print(f"Weather data filtered to: {daily_weather['DATE'].min()} to {daily_weather['DATE'].max()}")
print(f"Daily weather data: {daily_weather.shape}")

# Add date column to energy data for merging
df['DATE'] = df['INTERVAL_DATE'].dt.date

# Merge daily weather with energy data  
print("Merging daily weather data with energy data...")
df = df.merge(daily_weather, on='DATE', how='left')

# Check merge success
weather_cols = [col for col in df.columns if any(w in col for w in ['daily_temp', 'RH_mean', 'WINDSPD_mean', 'GLOBAL_mean', 'RAIN_sum', 'PRESS_mean'])]
print(f"Weather merge complete. Added {len(weather_cols)} weather columns")
print(f"Weather columns: {weather_cols[:5]}...")  # Show first 5 weather columns

# Check for missing weather data and clean up
missing_weather = df[weather_cols].isnull().sum().sum()
if missing_weather > 0:
    print(f"Found {missing_weather} missing weather values")
    
    # Show which dates have missing weather data
    missing_dates = df[df[weather_cols].isnull().any(axis=1)]['DATE'].unique()
    print(f"Missing weather dates: {sorted(missing_dates)}")
    
    # Drop records with missing weather data
    records_before = len(df)
    df = df.dropna(subset=weather_cols)
    records_after = len(df)
    
    print(f"Dropped {records_before - records_after} records with missing weather data")
    print(f"Dataset size: {records_before} → {records_after} records")
    print("Dataset is now clean with no missing weather values")
else:
    print("No missing weather data")

print("Weather data merge and cleaning complete!")

Loading weather data...
Energy data date range: 2022-01-01 to 2024-09-02
Weather data filtered to: 2022-01-01 to 2024-09-02
Daily weather data: (976, 15)
Merging daily weather data with energy data...


Weather merge complete. Added 8 weather columns
Weather columns: ['daily_temp_avg', 'daily_temp_max', 'daily_temp_min', 'RH_mean', 'WINDSPD_mean']...
Found 1610 missing weather values
Missing weather dates: [datetime.date(2023, 8, 1), datetime.date(2023, 8, 2)]
Dropped 230 records with missing weather data
Dataset size: 109189 → 108959 records
Dataset is now clean with no missing weather values
Weather data merge and cleaning complete!


In [5]:
# Basic Power Metrics for Clustering
print("CALCULATING BASIC POWER METRICS FOR CLUSTERING")

# Filter to only energy time columns (exclude weather columns)
time_cols_energy_only = [col for col in df.columns 
                        if '.' in col and '-' in col 
                        and not any(weather_var in col for weather_var in 
                                   ['TEMP_', 'RH_', 'WINDSPD_', 'WINDIR_', 'GLOBAL_', 'UVA_', 'UVB_', 'RAIN_', 'PRESS_', 'MAXGUST_'])]

print(f"Using {len(time_cols_energy_only)} energy columns for power metrics")

def calculate_household_power_metrics(df, time_cols_energy):
    all_households = df['ICP_IDENTIFIER'].unique()
    household_metrics = []
    
    print(f"Processing {len(all_households)} households...")
    
    for i, icp in enumerate(all_households):
        
        house_data = df[df['ICP_IDENTIFIER'] == icp]
        
        # Flattening - energy columns only
        all_power_values = house_data[time_cols_energy].values.flatten()
        all_power_values = all_power_values[~np.isnan(all_power_values)]
        
        if len(all_power_values) == 0:
            continue
        
        # Core power metrics
        baseload = np.percentile(all_power_values, 5)
        peak = np.max(all_power_values)
        avg_power = np.mean(all_power_values)
        median_power = np.median(all_power_values)
        power_std = np.std(all_power_values)
        
        # Key ratios for clustering
        peak_to_avg_ratio = peak / avg_power if avg_power > 0 else 0
        load_factor = avg_power / peak if peak > 0 else 0
        coefficient_of_variation = power_std / avg_power if avg_power > 0 else 0
        
        # Store metrics
        household_metrics.append({
            'ICP_IDENTIFIER': icp,
            'num_rooms': house_data['Rooms'].iloc[0] if 'Rooms' in house_data.columns else np.nan,
            'baseload_kW': baseload,
            'peak_kW': peak,
            'avg_power_kW': avg_power,
            'median_power_kW': median_power,
            'std_power_kW': power_std,
            'peak_to_avg_ratio': peak_to_avg_ratio,
            'load_factor': load_factor,
            'coefficient_of_variation': coefficient_of_variation
        })
    
    return pd.DataFrame(household_metrics)

# Calculate metrics
household_features = calculate_household_power_metrics(df, time_cols_energy_only)

# Quick validation
print(f"Baseload range: {household_features['baseload_kW'].min():.2f}-{household_features['baseload_kW'].max():.2f} kW")
print(f"Peak range: {household_features['peak_kW'].min():.1f}-{household_features['peak_kW'].max():.1f} kW")
print(f"Average range: {household_features['avg_power_kW'].min():.2f}-{household_features['avg_power_kW'].max():.2f} kW")
features_step1 = household_features.copy()

CALCULATING BASIC POWER METRICS FOR CLUSTERING
Using 48 energy columns for power metrics
Processing 118 households...
Baseload range: 0.00-5.40 kW
Peak range: 0.3-55.6 kW
Average range: 0.14-18.74 kW


In [6]:
# Weather Sensitivity Features
print("Calculating weather sensitivity features")

def calculate_weather_sensitivity_features(df, time_cols_energy):

    # Calculate daily energy consumption for each household using Total_Consumption
    print("Calculating daily energy consumption...")
    df['DATE'] = df['INTERVAL_DATE'].dt.date
    daily_energy = df.groupby(['ICP_IDENTIFIER', 'DATE'])['Total_Consumption'].sum().reset_index()
    daily_energy.rename(columns={'Total_Consumption': 'daily_total_kWh'}, inplace=True)
    
    # Get weather columns that are already in df
    weather_cols = [col for col in df.columns if any(w in col for w in ['daily_temp', 'RH_mean', 'WINDSPD_mean', 'GLOBAL_mean', 'RAIN_sum', 'PRESS_mean'])]
    
    # Merge energy with weather (weather is already in df)
    weather_daily = df[['DATE'] + weather_cols].drop_duplicates(subset=['DATE'])
    daily_merged = daily_energy.merge(weather_daily, on='DATE', how='inner')
    
    print(f"Merged data: {len(daily_merged)} daily records for weather-energy analysis")
    
    # Calculate temperature thresholds using percentile method
    temp_data = daily_merged['daily_temp_avg']
    unique_dates = daily_merged['DATE'].nunique()  # unique days only
    temp_min = temp_data.min()
    temp_max = temp_data.max()
    temp_mean = temp_data.mean()
    temp_median = temp_data.median()
    temp_std = temp_data.std()
    
    print(f"\n DUNEDIN TEMPERATURE DATA ANALYSIS:")
    print(f"  Temperature range: {temp_min:.1f}°C to {temp_max:.1f}°C")
    print(f"  Mean temperature: {temp_mean:.1f}°C")
    print(f"  Median temperature: {temp_median:.1f}°C")
    print(f"  Standard deviation: {temp_std:.1f}°C")
    print(f"  Analysis period: {unique_dates} unique days")
    
    # Percentile method for thresholds
    cold_threshold = np.percentile(temp_data, 25)  # 25th percentile
    warm_threshold = np.percentile(temp_data, 75)  # 75th percentile
    
    print(f"\n  CALCULATED TEMPERATURE THRESHOLDS:")
    print(f"  Cold threshold: {cold_threshold:.1f}°C (25th percentile)")
    print(f"  Warm threshold: {warm_threshold:.1f}°C (75th percentile)")
    
    # Show distribution using unique days only
    unique_temp_data = daily_merged.drop_duplicates('DATE')['daily_temp_avg']
    cold_days_count = (unique_temp_data < cold_threshold).sum()
    mild_days_count = ((unique_temp_data >= cold_threshold) & (unique_temp_data <= warm_threshold)).sum()
    warm_days_count = (unique_temp_data > warm_threshold).sum()
    
    print(f"  Cold days (< {cold_threshold:.1f}°C): {cold_days_count} days ({cold_days_count/len(unique_temp_data)*100:.1f}%)")
    print(f"  Mild days ({cold_threshold:.1f}°C - {warm_threshold:.1f}°C): {mild_days_count} days ({mild_days_count/len(unique_temp_data)*100:.1f}%)")
    print(f"  Warm days (> {warm_threshold:.1f}°C): {warm_days_count} days ({warm_days_count/len(unique_temp_data)*100:.1f}%)")
    
    # Calculate weather sensitivity for each household
    all_households = daily_merged['ICP_IDENTIFIER'].unique()
    weather_features = []
    
    print(f"Processing weather sensitivity for {len(all_households)} households...")
    
    for i, icp in enumerate(all_households):
        house_data = daily_merged[daily_merged['ICP_IDENTIFIER'] == icp].copy()
        
        # Weather variables for analysis
        temp_avg = house_data['daily_temp_avg']
        energy = house_data['daily_total_kWh']
        
        # Temperature sensitivity analysis
        temp_correlation = np.corrcoef(temp_avg, energy)[0, 1] if len(temp_avg) > 1 else 0
        
        # Heating patterns (using calculated thresholds)
        cold_days = house_data[temp_avg < cold_threshold]  # Cold days
        mild_days = house_data[(temp_avg >= cold_threshold) & (temp_avg <= warm_threshold)]  # Mild days
        
        cold_avg_energy = cold_days['daily_total_kWh'].mean() if len(cold_days) > 0 else 0
        mild_avg_energy = mild_days['daily_total_kWh'].mean() if len(mild_days) > 0 else 0
        
        # Weather sensitivity score
        heating_sensitivity = (cold_avg_energy - mild_avg_energy) if mild_avg_energy > 0 else 0
        
        # Seasonal patterns (month-based)
        house_data['month'] = pd.to_datetime(house_data['DATE']).dt.month
        
        # Winter (Jun-Aug), Spring (Sep-Nov), Summer (Dec-Feb), Autumn (Mar-May)
        winter_months = [6, 7, 8]
        spring_months = [9, 10, 11] 
        summer_months = [12, 1, 2]
        autumn_months = [3, 4, 5]
        
        winter_energy = house_data[house_data['month'].isin(winter_months)]['daily_total_kWh'].mean()
        spring_energy = house_data[house_data['month'].isin(spring_months)]['daily_total_kWh'].mean()
        summer_energy = house_data[house_data['month'].isin(summer_months)]['daily_total_kWh'].mean()
        autumn_energy = house_data[house_data['month'].isin(autumn_months)]['daily_total_kWh'].mean()
        
        # Seasonal variability
        seasonal_energies = [winter_energy, spring_energy, summer_energy, autumn_energy]
        seasonal_energies = [x for x in seasonal_energies if not np.isnan(x)]
        seasonal_range = (max(seasonal_energies) - min(seasonal_energies)) if len(seasonal_energies) > 1 else 0
        
        # Store weather sensitivity features 
        weather_features.append({
            'ICP_IDENTIFIER': icp,
            
            # Core temperature sensitivity (most important)
            'temperature_correlation': temp_correlation,
            'heating_sensitivity_kWh': heating_sensitivity,
            'seasonal_range_kWh': seasonal_range,
            
            # Energy consumption patterns
            'cold_days_avg_kWh': cold_avg_energy,
            
            # Data quality check
            'weather_analysis_days': len(house_data)
        })
    
    return pd.DataFrame(weather_features)

# Calculate weather sensitivity features
weather_sensitivity_features = calculate_weather_sensitivity_features(df, time_cols_energy_only)

# Merge with existing features - only power metrics
complete_features = features_step1.merge(
    weather_sensitivity_features, 
    on='ICP_IDENTIFIER', 
    how='left'
)

print(f"\nWeather Sensitivity Summary")
print(f"Weather features calculated for {len(weather_sensitivity_features)} households")

# Weather sensitivity insights
if len(weather_sensitivity_features) > 0:
    print(f"\nTemperature Correlation Range: {weather_sensitivity_features['temperature_correlation'].min():.3f} to {weather_sensitivity_features['temperature_correlation'].max():.3f}")
    print(f"Heating Sensitivity Range: {weather_sensitivity_features['heating_sensitivity_kWh'].min():.2f} to {weather_sensitivity_features['heating_sensitivity_kWh'].max():.2f} kWh")
    print(f"Seasonal Variability Range: {weather_sensitivity_features['seasonal_range_kWh'].min():.2f} to {weather_sensitivity_features['seasonal_range_kWh'].max():.2f} kWh")

    # Identify weather-sensitive vs weather-stable accommodations
    high_temp_sensitivity = (abs(weather_sensitivity_features['temperature_correlation']) > 0.3).sum()
    high_seasonal_variation = (weather_sensitivity_features['seasonal_range_kWh'] > weather_sensitivity_features['seasonal_range_kWh'].median()).sum()

    print(f"\nWeather Sensitivity Patterns:")
    print(f"  High temperature sensitivity (|corr| > 0.3): {high_temp_sensitivity} accommodations ({high_temp_sensitivity/len(weather_sensitivity_features)*100:.1f}%)")
    print(f"  High seasonal variation: {high_seasonal_variation} accommodations ({high_seasonal_variation/len(weather_sensitivity_features)*100:.1f}%)")

# Save complete features with weather sensitivity
features_step2 = complete_features.copy()

Calculating weather sensitivity features
Calculating daily energy consumption...
Merged data: 108959 daily records for weather-energy analysis

 DUNEDIN TEMPERATURE DATA ANALYSIS:
  Temperature range: 3.1°C to 25.1°C
  Mean temperature: 12.0°C
  Median temperature: 12.0°C
  Standard deviation: 3.7°C
  Analysis period: 969 unique days

  CALCULATED TEMPERATURE THRESHOLDS:
  Cold threshold: 9.2°C (25th percentile)
  Warm threshold: 14.6°C (75th percentile)
  Cold days (< 9.2°C): 241 days (24.9%)
  Mild days (9.2°C - 14.6°C): 486 days (50.2%)
  Warm days (> 14.6°C): 242 days (25.0%)
Processing weather sensitivity for 118 households...

Weather Sensitivity Summary
Weather features calculated for 118 households

Temperature Correlation Range: -0.838 to -0.257
Heating Sensitivity Range: 0.58 to 200.81 kWh
Seasonal Variability Range: 1.29 to 392.71 kWh

Weather Sensitivity Patterns:
  High temperature sensitivity (|corr| > 0.3): 116 accommodations (98.3%)
  High seasonal variation: 59 accommo

In [7]:
# Time Pattern Features 
print("Calculating Time Pattern Features")

def calculate_time_pattern_features(df):
   
    print("Processing time patterns...")
    
    # Data is in wide format 
    # Use existing DATETIME column
    df['datetime'] = pd.to_datetime(df['DATETIME'])
    df['date'] = df['datetime'].dt.date
    df['dayofweek'] = df['datetime'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['is_weekend'] = df['dayofweek'].isin([5, 6])  # Saturday, Sunday
    
    # Get all half-hourly columns (48 periods)
    half_hourly_cols = [col for col in df.columns if '.' in col and '-' in col]
    print(f"Found {len(half_hourly_cols)} half-hourly columns")
    
    # Define time periods by mapping half-hourly columns to periods
    def get_time_period_cols():
        night_cols = []    # 11pm-6am (23:00-06:00)
        morning_cols = []  # 6am-10am  (06:00-10:00)  
        daytime_cols = []  # 10am-5pm  (10:00-17:00)
        evening_cols = []  # 5pm-11pm  (17:00-23:00)
        
        for col in half_hourly_cols:
            # Extract start hour from column name 
            start_time = col.split('-')[0]
            hour = int(start_time.split('.')[0])
            
            if hour >= 23 or hour < 6:  # 11pm-6am
                night_cols.append(col)
            elif 6 <= hour < 10:  # 6am-10am
                morning_cols.append(col)
            elif 10 <= hour < 17:  # 10am-5pm
                daytime_cols.append(col)
            elif 17 <= hour < 23:  # 5pm-11pm
                evening_cols.append(col)
        
        return night_cols, morning_cols, daytime_cols, evening_cols
    
    night_cols, morning_cols, daytime_cols, evening_cols = get_time_period_cols()
    
    print("Time period definitions:")
    print("  Night (Sleep): 11pm-6am")
    print("  Morning: 6am-10am") 
    print("  Daytime: 10am-5pm")
    print("  Evening: 5pm-11pm")
    
    # Calculate features for each household
    all_households = df['ICP_IDENTIFIER'].unique()
    time_features = []
    
    print(f"Processing time patterns for {len(all_households)} households...")
    
    for i, icp in enumerate(all_households):
        house_data = df[df['ICP_IDENTIFIER'] == icp].copy()
        
        # Convert half-hourly kW values back to kWh for time pattern analysis
        half_hourly_kwh = house_data[half_hourly_cols] / 2
        
        # Calculate time-of-day energy consumption by summing relevant half-hourly columns
        night_consumption = half_hourly_kwh[[col for col in half_hourly_cols if col in night_cols]].sum(axis=1).sum()
        morning_consumption = half_hourly_kwh[[col for col in half_hourly_cols if col in morning_cols]].sum(axis=1).sum()
        daytime_consumption = half_hourly_kwh[[col for col in half_hourly_cols if col in daytime_cols]].sum(axis=1).sum()
        evening_consumption = half_hourly_kwh[[col for col in half_hourly_cols if col in evening_cols]].sum(axis=1).sum()
        
        total_consumption = night_consumption + morning_consumption + daytime_consumption + evening_consumption
        
        # Calculate ratios 
        night_ratio = (night_consumption / total_consumption) if total_consumption > 0 else 0
        morning_ratio = (morning_consumption / total_consumption) if total_consumption > 0 else 0
        daytime_ratio = (daytime_consumption / total_consumption) if total_consumption > 0 else 0
        evening_ratio = (evening_consumption / total_consumption) if total_consumption > 0 else 0
        
        # Daily consumption consistency using Total_Consumption column (kWh)
        daily_totals = house_data['Total_Consumption']
        daily_mean = daily_totals.mean()
        daily_std = daily_totals.std()
        daily_consistency = (daily_std / daily_mean) if daily_mean > 0 else 0
        
        # Weekday vs Weekend patterns (using Total_Consumption kWh)
        weekday_avg = house_data[~house_data['is_weekend']]['Total_Consumption'].mean()
        weekend_avg = house_data[house_data['is_weekend']]['Total_Consumption'].mean()
        weekday_weekend_ratio = (weekday_avg / weekend_avg) if weekend_avg > 0 else 0
        
        # Usage concentration across 48 half-hourly periods (using kWh)
        half_hourly_avgs = half_hourly_kwh.mean()
        total_avg = half_hourly_avgs.sum()
        half_hourly_ratios = half_hourly_avgs / total_avg if total_avg > 0 else pd.Series([0]*len(half_hourly_cols))
        usage_concentration = (half_hourly_ratios ** 2).sum()
        
        # Get total days for data quality tracking
        total_days = len(daily_totals)
        
        # Store time pattern features
        time_features.append({
            'ICP_IDENTIFIER': icp,
            
            # Time-of-day patterns (4 features)
            'night_usage_ratio': night_ratio,
            'morning_usage_ratio': morning_ratio,
            'daytime_usage_ratio': daytime_ratio,
            'evening_usage_ratio': evening_ratio,
            
            # Behavioral consistency (2 features)
            'daily_usage_consistency': daily_consistency,
            'weekday_weekend_ratio': weekday_weekend_ratio,
            
            # Advanced patterns (1 feature)
            'usage_concentration': usage_concentration,
            
            # Data quality
            'time_analysis_days': total_days
        })
    
    return pd.DataFrame(time_features)

# Calculate time pattern features
time_pattern_features = calculate_time_pattern_features(df)

print(f"\nTIME PATTERN SUMMARY")
print(f"Time features calculated for {len(time_pattern_features)} households")

# Time pattern insights
if len(time_pattern_features) > 0:
    print(f"\nTime Energy Distribution:")
    print(f"  Night energy: {time_pattern_features['night_usage_ratio'].mean():.1%} ± {time_pattern_features['night_usage_ratio'].std():.1%}")
    print(f"  Morning energy: {time_pattern_features['morning_usage_ratio'].mean():.1%} ± {time_pattern_features['morning_usage_ratio'].std():.1%}")
    print(f"  Daytime energy: {time_pattern_features['daytime_usage_ratio'].mean():.1%} ± {time_pattern_features['daytime_usage_ratio'].std():.1%}")
    print(f"  Evening energy: {time_pattern_features['evening_usage_ratio'].mean():.1%} ± {time_pattern_features['evening_usage_ratio'].std():.1%}")
    
    print(f"\nBehavioral Patterns:")
    print(f"  Daily consistency (CV): {time_pattern_features['daily_usage_consistency'].mean():.3f} ± {time_pattern_features['daily_usage_consistency'].std():.3f}")
    print(f"  Weekday/Weekend ratio: {time_pattern_features['weekday_weekend_ratio'].mean():.2f} ± {time_pattern_features['weekday_weekend_ratio'].std():.2f}")
    print(f"  Usage concentration: {time_pattern_features['usage_concentration'].mean():.3f} ± {time_pattern_features['usage_concentration'].std():.3f}")
    
    features_step3 = time_pattern_features.copy()


Calculating Time Pattern Features
Processing time patterns...
Found 48 half-hourly columns
Time period definitions:
  Night (Sleep): 11pm-6am
  Morning: 6am-10am
  Daytime: 10am-5pm
  Evening: 5pm-11pm
Processing time patterns for 118 households...

TIME PATTERN SUMMARY
Time features calculated for 118 households

Time Energy Distribution:
  Night energy: 25.2% ± 4.9%
  Morning energy: 15.4% ± 1.8%
  Daytime energy: 28.9% ± 4.3%
  Evening energy: 30.5% ± 2.3%

Behavioral Patterns:
  Daily consistency (CV): 0.667 ± 0.150
  Weekday/Weekend ratio: 1.02 ± 0.05
  Usage concentration: 0.022 ± 0.002


In [8]:
# Room-Normalized Features
print("=== CALCULATING ROOM-NORMALIZED FEATURES ===")

def calculate_room_normalized_features(df):
    """Calculate per-room consumption and efficiency metrics for each household"""
    
    print("Processing room-normalized consumption patterns...")
    
    # Get room information for each household
    room_info = df.groupby('ICP_IDENTIFIER')['Rooms'].first().reset_index()
    print(f"Room distribution: {room_info['Rooms'].value_counts().sort_index().to_dict()}")
    
    # Calculate room-normalized features for each household
    all_households = df['ICP_IDENTIFIER'].unique()
    room_features = []
    
    # Get all half-hourly columns 
    half_hourly_cols = [col for col in df.columns if '.' in col and '-' in col and col != 'INTERVAL_DATE']
    
    for i, icp in enumerate(all_households):
        house_data = df[df['ICP_IDENTIFIER'] == icp].copy()
        
        # Get room count
        rooms = max(house_data['Rooms'].iloc[0], 1)  # Avoid division by zero
        
        # 1. Average power per room (kW/room) - baseline energy intensity
        avg_power_per_room = house_data[half_hourly_cols].mean().mean() / rooms
        
        # 2. Peak power per room (kW/room) - maximum demand capacity
        daily_peaks = house_data.groupby('INTERVAL_DATE')[half_hourly_cols].max().max(axis=1)
        peak_power_per_room = daily_peaks.mean() / rooms
        
        # 3. Power variability per room (CV) - usage pattern consistency
        daily_avg_power = house_data.groupby('INTERVAL_DATE')[half_hourly_cols].mean().mean(axis=1)
        daily_power_per_room = daily_avg_power / rooms
        power_variability_per_room = (daily_power_per_room.std() / avg_power_per_room) if avg_power_per_room > 0 else 0
        
        # Store the essential room-normalised features
        room_features.append({
            'ICP_IDENTIFIER': icp,
            'rooms': rooms,
            
            # Core room-normalised features (3 features)
            'avg_power_per_room': avg_power_per_room,
            'peak_power_per_room': peak_power_per_room,
            'power_variability_per_room': power_variability_per_room
        })
        
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{len(all_households)} households...")
    
    room_features_df = pd.DataFrame(room_features)

    # Merge with existing features
    print("Merging room-normalized features with existing features...")
    
    # Start with weather features as base
    complete_features = features_step2.copy()
    
    # Add time pattern features 
    complete_features = complete_features.merge(
        features_step3[['ICP_IDENTIFIER'] + [col for col in features_step3.columns if col != 'ICP_IDENTIFIER']], 
        on='ICP_IDENTIFIER', 
        how='left'
    )
    
    # Add room-normalised features (excluding rooms and ICP_IDENTIFIER)
    room_feature_cols = ['ICP_IDENTIFIER', 'avg_power_per_room', 'peak_power_per_room', 'power_variability_per_room']
    complete_features = complete_features.merge(
        room_features_df[room_feature_cols], 
        on='ICP_IDENTIFIER', 
        how='left'
    )
    
    return room_features_df, complete_features

# Calculate room-normalised features
room_features, final_complete_features = calculate_room_normalized_features(df)

print(f"\nROOM-NORMALIZED SUMMARY")

# Round all numeric columns to 2 decimals
numeric_cols = final_complete_features.select_dtypes(include=[np.number]).columns
final_complete_features[numeric_cols] = final_complete_features[numeric_cols].round(2)

# Save complete feature set with a clean name
final_complete_features.to_csv('final_features.csv', index=False)
print(f"\n Final complete feature set saved to 'final_features.csv'")
print(f"Total features: {len(final_complete_features.columns) - 1}")
print(f"Total households: {len(final_complete_features)}")  # -1 for ICP_IDENTIFIER


=== CALCULATING ROOM-NORMALIZED FEATURES ===
Processing room-normalized consumption patterns...
Room distribution: {1: 2, 2: 13, 3: 8, 4: 24, 5: 37, 6: 31, 7: 1, 9: 1, 11: 1}
Processed 100/118 households...
Merging room-normalized features with existing features...

ROOM-NORMALIZED SUMMARY

 Final complete feature set saved to 'final_features.csv'
Total features: 25
Total households: 118
